# Базы данных

## Лекция 9

### Работа с базами данных в Python. Курсоры. ORM

МФТИ ФПМИ 2021

[Python DB API 2.0](https://www.python.org/dev/peps/pep-0249/) - стандарт интерфейсов для пакетов, работающих с БД

Для PostgreSQL: [psycopg2](http://initd.org/psycopg/)

In [1]:
try:
    import psycopg2
except:
    !pip install psycopg2-binary

## Курсор

Вместо того чтобы сразу выполнять весь запрос, есть возможность настроить курсор, инкапсулирующий запрос, и затем получать результат запроса по нескольку строк за раз. Одна из причин так делать заключается в том, чтобы избежать переполнения памяти, когда результат содержит большое количество строк.

В PL/pgSQL:
```sql
name [ [ NO ] SCROLL ] CURSOR [ ( arguments ) ] FOR query;
```


```sql
DECLARE
    curs1 refcursor;
    curs2 CURSOR FOR SELECT * FROM tenk1;
    curs3 CURSOR (key integer) FOR SELECT * FROM tenk1 WHERE unique1 = key;
```

### Открывание курсора


```sql
OPEN unbound_cursorvar [ [ NO ] SCROLL ] FOR query;

OPEN curs1 FOR SELECT * FROM foo WHERE key = mykey;
OPEN curs2;
```

### Открывание курсора на EXECUTE


```sql
OPEN unbound_cursorvar [ [ NO ] SCROLL ] FOR EXECUTE query_string
                                     [ USING expression [, ... ] ];

OPEN curs1 FOR EXECUTE 'SELECT * FROM ' || quote_ident(tabname)
                                        || ' WHERE col1 = $1' USING keyvalue;
```

### Передача параметров в курсор

```sql
DECLARE
    -- Переменная обязательно вводится до курсора
    key integer;
    curs4 CURSOR FOR SELECT * FROM tenk1 WHERE unique1 = key;
    curs3 CURSOR (key integer) FOR SELECT * FROM tenk1 WHERE unique1 = key;
BEGIN
    key := 42;
    OPEN curs4;
    OPEN curs3(42);
```

Если необходимо освободить ресурсы до завершения транзакции или освободить переменную курсора для повторного открывания, можно закрыть уже открытый курсор:

```sql
CLOSE cursor;

CLOSE curs1;
```

## Использование курсоров

Как только курсор открыт, можно начинать работу с данными

## FETCH

FETCH извлекает строку из курсора в заданный таргет.

```sql
FETCH [ direction { FROM | IN } ] cursor INTO target;
```

В качестве direction могут использоваться: NEXT, FIRST, LAST, ABSOLUTE count, RELATIVE count и т.д.

```sql
FETCH curs1 INTO rowvar;
FETCH curs2 INTO foo, bar, baz;
FETCH LAST FROM curs3 INTO x, y;
FETCH RELATIVE -2 FROM curs4 INTO x;
```

In [2]:
# Соединение с уже существующей базой данных
conn =  psycopg2.connect(dbname="postgres", user="merkureva", password="1234", host="localhost")
# Открывание курсора для работы с базой
cur = conn.cursor()
# Отправка запроса в базу
cur.execute("select 'Hello, world!', 6 * 7")
# Получение одной строки с результатом запроса
print(cur.fetchone())
conn.close()

('Hello, world!', 42)


Не забываем закрывать соединение! <br>
**Connection - это ресурс, который может и закончиться.** <br>
В этом нам помогут контекстные менеджеры.

In [3]:
# Не делайте так. Используйте конфиг файлы
params = dict(dbname="postgres", user="merkureva", password="1234", host="localhost")

### Создадим таблички

In [4]:
with psycopg2.connect(**params) as conn:
    cur = conn.cursor()
    # Создадим схему для сегодняшней лекции
    cur.execute('DROP SCHEMA IF EXISTS lecture10 cascade;')
    cur.execute('CREATE SCHEMA lecture10;')
    
    # SQL диалект может немного меняться
    cur.execute('''
        CREATE TABLE lecture10.user_types (
            id SERIAL PRIMARY KEY,
            name VARCHAR(255)
        );
    ''')
    cur.execute('''
        CREATE TABLE lecture10.users (
            id SERIAL PRIMARY KEY,
            first_name VARCHAR(255),
            last_name VARCHAR(255),
            active BOOLEAN,
            profile INTEGER,
            FOREIGN KEY(profile) REFERENCES lecture10.user_types(id)
        );
    ''')

In [5]:
def add_user_types(cur):
    cur.execute('''INSERT INTO lecture10.user_types(name) VALUES ('Student')''')
    cur.execute('''INSERT INTO lecture10.user_types(name) VALUES ('Teacher')''')

def add_teachers(cur):
    cur.execute('''
        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('Maxim', 'Popov', '1', 
               (SELECT id FROM lecture10.user_types WHERE name = 'Teacher'))''')
    cur.execute('''
        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('Igor', 'Orlov', '1', 
               (SELECT id FROM lecture10.user_types WHERE name = 'Teacher'))''')

def add_students(cur):
    cur.execute('''
        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('Ivan', 'Ivanov', '0', 
               (SELECT id FROM lecture10.user_types WHERE name = 'Student'))''')
    cur.execute('''
        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('Petr', 'Petrov', '0', 
               (SELECT id FROM lecture10.user_types WHERE name = 'Student'))''')
    cur.execute('''
        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('Petr', 'Sidorov', '0', 
               (SELECT id FROM lecture10.user_types WHERE name = 'Student'))''')

#### Выполним запросы к базе

In [6]:
with psycopg2.connect(**params) as conn:

    cur = conn.cursor()
    
    # Функции добавления пользователей не вызывают commit
    add_user_types(cur)
    add_teachers(cur)
    add_students(cur)
    
    # Делаем commit для всех запросов добавления
    # Если забудем - то таблицы останутся пустыми
    conn.commit()

#### Попробуем теперь вычитать всех студентов

In [7]:
def select_all_students(conn):
    query = '''SELECT first_name
                    , last_name 
                 FROM lecture10.users 
                WHERE profile = (SELECT id
                                   FROM lecture10.user_types
                                  WHERE name = 'Student')'''
    cur = conn.cursor()
    cur.execute(query)

    # Получить все строчки результата выполнения query
    # Это не всегда быстро, особенно если много результатов
    # Кроме того большое потребление памяти, так как возвращается list
    rows = cur.fetchall() 

    # Как fetchall только задаем сколько хотим получить
    # rows = cur.fetchmany(10) 

    for row in rows:
        first_name, last_name = row
        print(first_name, last_name)
        
with psycopg2.connect(**params) as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


In [8]:
def select_all_students(conn):
    query = '''SELECT first_name
                    , last_name 
                 FROM lecture10.users 
                WHERE profile = (SELECT id
                                   FROM lecture10.user_types
                                  WHERE name = 'Student')'''
    cur = conn.cursor()
    cur.execute(query)

    # А лучше просто проитерироваться
    for row in cur:
        first_name, last_name = row
        print(first_name, last_name)
        
with psycopg2.connect(**params) as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov


Попробуем сделать учителя Maxim Popov студентом

In [9]:
with psycopg2.connect(**params) as conn:
    cur = conn.cursor()
    cur.execute(
        'SELECT * FROM lecture10.users'
    )
    print(cur.fetchall())

[(1, 'Maxim', 'Popov', True, 2), (2, 'Igor', 'Orlov', True, 2), (3, 'Ivan', 'Ivanov', False, 1), (4, 'Petr', 'Petrov', False, 1), (5, 'Petr', 'Sidorov', False, 1)]


In [10]:
def alter_teacher(conn):
    cur = conn.cursor()
    cur.execute('''
        UPDATE lecture10.users
           SET profile = (SELECT id
                            FROM lecture10.user_types
                           WHERE name = 'Student')
         WHERE first_name = 'Maxim'
           AND last_name = 'Popov'
    ''')

with  psycopg2.connect(**params) as conn:
    alter_teacher(conn)
# '''

Посмотрим на результат

In [11]:
with psycopg2.connect(**params) as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov
Maxim Popov


In [12]:
# Параметризируем запрос
def add_student(cur, first_name, last_name, active):
    query = """
        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('{}', '{}', '{}', 1)""".format(first_name, last_name, active)
    print(query)
    cur.execute(query)

In [13]:
with psycopg2.connect(**params) as conn:
    cur = conn.cursor()
    
    add_student(cur, 'Victor', 'Victorov', 1)
    add_student(cur, 'Sergey', 'Sergeev', 1)
    conn.commit()


        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('Victor', 'Victorov', '1', 1)

        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES ('Sergey', 'Sergeev', '1', 1)


In [14]:
with psycopg2.connect(**params) as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov
Maxim Popov
Victor Victorov
Sergey Sergeev


Where is my table, Lebowski???

<img src="https://imgs.xkcd.com/comics/exploits_of_a_mom.png">

## Что тогда делать?
https://www.psycopg.org/docs/usage.html#query-parameters

**Warning Never, never, NEVER use Python string concatenation (+) or string parameters interpolation (%) to pass variables to a SQL query string. Not even at gunpoint.**

The correct way to pass variables in a SQL command is using the second argument of the execute() method:

```python
SQL = "INSERT INTO authors (name) VALUES (%s);" # Note: no quotes
data = ("O'Reilly", )
cur.execute(SQL, data) # Note: no % operator
```

### Перепишем нашу функцию.

In [15]:
def add_student(cur, first_name, last_name, active):
    query = """
        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES (%s, %s, %s, 1)"""
    print(query)
    cur.execute(query, (first_name, last_name, active))

In [16]:
with psycopg2.connect(**params) as conn:
    cur = conn.cursor()
    
    add_student(cur, 'Anna', 'Chernova', True)
    conn.commit()


        INSERT INTO lecture10.users (first_name, last_name, active, profile)
        VALUES (%s, %s, %s, 1)


In [17]:
with psycopg2.connect(**params) as conn:
    select_all_students(conn)

Ivan Ivanov
Petr Petrov
Petr Sidorov
Maxim Popov
Victor Victorov
Sergey Sergeev
Anna Chernova


### Что осталось за бортом?

Как обычно, дальше спасает https://www.google.com/

# ORM

ORM (Object-Relational Mapping) — технология программирования, которая
связывает базы данных с концепциями объектно-ориентированных языков
программирования, создавая «виртуальную объектную базу данных». Существует
множество вариантов реализации этой технологии.

## Задача

Необходимо обеспечить работу с данными в терминах классов, а не таблиц данных и напротив, преобразовать термины и данные классов в данные, пригодные для хранения в СУБД. Необходимо также обеспечить интерфейс для CRUD-операций над данными. В общем, необходимо избавиться от необходимости писать SQL-код для взаимодействия в СУБД.

## Решение

Решение **проблемы хранения** существует — это реляционные СУБД. Но их использование для хранения объектно-ориентированных данных приводит к семантическому разрыву, заставляя программистов писать программное обеспечение, которое должно уметь как обрабатывать данные в объектно- ориентированном виде, так и уметь сохранить эти данные в реляционной форме.

Разработано множество пакетов, устраняющих необходимость в преобразовании объектов для хранения в реляционных базах данных.
С точки зрения программиста система должна выглядеть как постоянное хранилище объектов. Он может просто создавать объекты и работать с ними как обычно, а они автоматически будут сохраняться в реляционной базе данных.

## Peewee

http://docs.peewee-orm.com/en/latest/

In [18]:
try:
    from peewee import *
except:
    !pip install peewee
    from peewee import *

db = PostgresqlDatabase(database="postgres", user="merkureva", password="1234", host='localhost')

In [19]:
class Person(Model):
    name = CharField()
    birthday = DateField()
    is_relative = BooleanField()
    
    # другие типы полей:   https://peewee.readthedocs.io/en/latest/peewee/models.html#field-types-table
    
    class Meta:
        database = db # модель будет использовать базу данных 'postgres'

Инициализирующие аргументы:
- *null=False* – возможно ли хранение null-значений;
- *index=False* – создавать ли индекс для данного столбца в базе;
- *unique=False* – создавать ли уникальный индекс для данного столбца в базе.
- *verbose_name=None* – строка для человекопонятного представления поля;
- *help_text=None* – строка с вспомогательным текстом для поля;
- *db_column=None* – строка, явно задающая название столбца в базе для данного поля, используется например при работе с legacy-базой данных;
- *default=None* – значение по умолчанию для полей класса при инстанцировании;
- *choices=None* – список или кортеж двухэлементных кортежей, где первый элемент – значение для базы, второй – отображаемое значение (аналогично джанге);
- *primary_key=False* – использовать ли данное поле, как первичный ключ;

In [20]:
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()
    
    class Meta:
        database = db # модель будет использовать базу данных 'postgres'

In [21]:
# Создаем таблицы в базе со всеми нужными колонками, ключами и сиквенсами (если они нужны)
Person.create_table()
Pet.create_table()

In [22]:
from datetime import date


uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15),
is_relative=True)
uncle_bob.save() # cохраним Боба в базе данных

grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1),
is_relative=True)
herb = Person.create(name='Herb', birthday=date(1950, 5, 5),
is_relative=False)

grandma.name = 'Grandma L.'

grandma.save() # обновим запись grandma

1

In [23]:
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [24]:
grandma = Person.select().where(Person.name == 'Grandma L.').get()
grandma.name

'Grandma L.'

In [25]:
grandma = Person.get(Person.name == 'Grandma L.')
grandma.name

'Grandma L.'

In [26]:
for person in Person.select():
    print(person.name, person.is_relative)

Bob True
Herb False
Grandma L. True


In [27]:
for person in Person.select():
    print(person.name, person.pets.count(), 'pets')
    for pet in person.pets:
        print (' ', pet.name, pet.animal_type)

Bob 1 pets
  Kitty cat
Herb 3 pets
  Fido dog
  Mittens cat
  Mittens Jr cat
Grandma L. 0 pets


In [28]:
herb_mittens.delete_instance()
herb_fido.owner = uncle_bob
herb_fido.save()

1

In [29]:
# агрегация значений
for person in Person.select():
    print(person.name, person.pets.count(), 'pets')
    for pet in person.pets:
        print (' ', pet.name, pet.animal_type)

Bob 2 pets
  Kitty cat
  Fido dog
Herb 1 pets
  Mittens Jr cat
Grandma L. 0 pets


In [30]:
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [31]:
query = (Pet
         .select(Pet, Person)
         .join(Person)
         .where(Pet.animal_type == 'cat'))

for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [32]:
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print(pet.name)

Kitty
Fido


In [33]:
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name)

Bob
Herb
Grandma L.


In [35]:
# Используем сложное условие в запросе
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
         .select()
         .where((Person.birthday < d1940) | (Person.birthday > d1960)))

for person in query:
    print(person.name, person.birthday)

Bob 1960-01-15
Grandma L. 1935-03-01


In [37]:
query = (Person
         .select()
         .where(Person.birthday.between(d1940, d1960)))

for person in query:
    print(person.name, person.birthday)

Herb 1950-05-05


In [38]:
# соединение нескольких таблиц
query = (Person
         .select(Person, fn.COUNT(Pet.id).alias('pet_count'))
         .join(Pet, JOIN.LEFT_OUTER)  # не исключаем людей без животных
         .group_by(Person)
         .order_by(Person.name))

for person in query:
    print(person.name, person.pet_count, 'pets')

Bob 2 pets
Grandma L. 0 pets
Herb 1 pets


In [39]:
db.close()

True

## ORM: плюсы и минусы

ORM, по идее, должен избавить нас от написания SQL запросов и, в идеале, вообще абстрагировать от базы данных (от способа хранения данных), чтобы мы могли работать с классами, в той или иной степени выражающими объекты бизнес-логики, не задаваясь вопросом, в каких таблицах всё это по факту лежит.

Для примера возьмем две таблицы: книги и авторы книг, отношение многие-ко- многим (у книг может быть много авторов, у авторов может быть много книг). Т.е. в базе это будут books, authors и связующая таблица author_book:

```sql
 CREATE TABLE authors (
    id serial
  , name varchar(1000) not null
  , PRIMARY KEY (id)
);
CREATE TABLE books (
    id serial
  , name VARCHAR (1000) not null
  , text text not null
  , PRIMARY KEY (id)
);
CREATE TABLE author_book (
    author_id bigint REFERENCES authors(id)
  , book_id bigint REFERENCES books(id)
  , PRIMARY KEY (author_id, book_id)
);
```

## ORM: плюсы и минусы

### Кейс 1: создание записей

- Pure SQL
    - Piece of cake. Простые CREATE и INSERT
    - Много писанины
    - Нужно знать синтаксис SQL
- ORM
    - Piece of cake. Создаём нужные классы, наполняем
    - Много писанины
    - Нужно разобраться с ORM

## ORM: плюсы и минусы

### Кейс 2: обновление записей

- Pure SQL
    - Piece of cake. Просто UPDATE
- ORM
    - Piece of cake. Пользуемся нужным методом (как с именем бабули ранее)

## ORM: плюсы и минусы

### Кейс 3: простая выборка
-  Pure SQL
    - Piece of cake. Запрос с агрегацией:
    ```sql
      select
        b.id as book_id
        , b.name as book_name
        , json_agg(a.name) as authors
      from
        books b
      inner join
        author_book ab
          on b.id = ab.book_id
      inner join
        authors a
          on ab.author_id = a.id
      group by
        b.id;
```
- ORM
    - Piece of cake (так ли это?)
        - Проходим циклом и находим нужное
        - Будет несколько запросов: выборка всех книг и для каждой выборка автора
        - С ростом количества данных будет выполняться всё дольше
        - Вытянутся все поля, а не только нужные; придётся разбираться с SQL

## ORM: плюсы и минусы

### Кейс 4: сложное обновление

- Pure SQL
    - Piece of cake. Простой подзапрос
    ```sql
      UPDATE authors
      SET name = 'Лев Толстой'
      WHERE id in (
          SELECT
            id
          FROM
            authors
          ORDER BY
            id DESC
          LIMIT 2 );
    ```
- ORM
    - Долгое изучение мануалов
    - Никаких подзапросов
    - Неоптимально с точки зрения БД